In [ ]:
import json
import numpy as np
import parmed as pmd
import matplotlib.pyplot as plt

from paprika.analysis import fe_calc
from paprika.io import load_restraints, PaprikaEncoder
from paprika.restraints.utils import extract_guest_restraints

## Calculate attachment free-energy

In [ ]:
method = "ti-block"
restraints = load_restraints(filepath="../boresch/boresch_restraints.json")

In [ ]:
free_energy = fe_calc()
free_energy.topology = "system.pdb"
free_energy.trajectory = "production.dcd"
free_energy.path = "../attach"
free_energy.restraint_list = restraints
free_energy.collect_data()
free_energy.fractions = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
free_energy.methods = [method]
free_energy.ti_matrix = "diagonal"
free_energy.bootcycles = 1000
free_energy.compute_free_energy()
free_energy.compute_ref_state_work(restraints, state="initial")

In [ ]:
results = free_energy.results
with open("results_attach.json", "w") as f:
    dumped = json.dumps(results, cls=PaprikaEncoder)
    f.write(dumped)

## Print free-energy results

In [ ]:
def print_binding(results, method="ti-block"):
    fe_a   = -1*results["attach"][method]["fe"].magnitude
    sem_a  = results["attach"][method]["sem"].magnitude
    fe_ref = -1*results["ref_state_work"].magnitude

    print(f"Analysis - {method}")
    print("-" * 25)
    print(f"Attach free-energy    = {fe_a:6.2f} +/- {sem_a:0.2f} kcal/mol")
    print(f"Reference free-energy = {fe_ref:6.2f} kcal/mol")
    print("")


print_binding(results, method=method)

In [ ]:
def get_fe_convergence(results, method="ti-block"):
    convergence = {}

    # Free energy
    attach  = results["attach"][method]["fraction_fe"]
    convergence["fe_a"] = -1 * np.array([attach[i].magnitude for i in attach])
    convergence["fractions"] = np.array([i for i in attach])

    # Error
    attach  = results["attach"][method]["fraction_sem"]
    convergence["sem_a"] = np.array([attach[i].magnitude for i in attach])

    return convergence


convergence = get_fe_convergence(results, method=method)

## Plot free-energy profile and convergence

In [ ]:
attach_string = (
    "0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 18.10 24.40 37.00 49.60 74.80 100.00"
)
attach_fractions = [float(i) / 100 for i in attach_string.split()]

plt.figure(dpi=100, figsize=(12,4))
plt.subplot(1,2,1)
plt.errorbar(
    attach_fractions,
    results["attach"]["ti-block"]["fe_matrix"][0, :],
    yerr=results["attach"]["ti-block"]["sem_matrix"][0, :],
    fmt="-o",
)
plt.xlabel("lambda", fontsize=12)
plt.ylabel(r"$\Delta G$ (kcal/mol)", fontsize=12)
plt.title("FE over lambda", fontsize=14)

plt.subplot(1,2,2)
plt.errorbar(
    convergence["fractions"], convergence["fe_a"], yerr=convergence["sem_a"], fmt="o-"
)
plt.xlabel("fraction", fontsize=12)
plt.ylabel(r"$\Delta G$ (kcal/mol)", fontsize=12)
plt.title("FE convergence", fontsize=14)